# Predicting Return on Investment (ROI) of Consumer Loans

**Andrew Nicholls** | Email: andrew.s.nicholls@gmail.com | [Github](https://github.com/Booleans)

If you are viewing this notebook on Github I recommend using the following nbviewer link instead to ensure proper formatting and working interactive charts.

[nbviewer link](https://nbviewer.jupyter.org/github/Booleans/Lending-Club-Loan-Analysis/blob/master/Loan_Analysis_Regression.ipynb)

## Summary

**Contents:**

1. [Problem Definition and Background Information](#1)
2. [Data Preparation: Wrangling, Cleaning, and Feature Extraction](#2)
3. [Exploratory Data Analysis](#3)
4. [Machine Learning Models](#4)
5. [Results and Findings](#4)
<a id='1'></a>

# 1. Introduction

### Problem Definition

LendingClub Corporation operates as an online marketplace that connects borrowers and investors in the United States. Its marketplace facilitates various types of loan products for consumers and small businesses, including unsecured personal loans, super prime consumer loans, unsecured education and patient finance loans, and unsecured small business loans. The company also offers investors an opportunity to invest in a range of loans based on term and credit characteristics. However, many loans issued through Lending Club end up being defaulted on by the borrower. The goal of this notebook is to examine the historical loan data available in order to generate a model to predict the return on investment that a new loan will generate.

### Files Provided

Lending Club provides CSV files of historical data for its loans. These files contain complete loan data for all loans issued through the time period stated, including the loan status (Current, Late, Fully Paid, etc.) and latest payment information. The files can be acquired on the [Lending Club Statistics Page](https://www.lendingclub.com/info/download-data.action). As of the creation of this project the latest data available from Lending Club was for Q1 2018.

For information on the definitions of fields contained within the historical data please see the [Lending Club Data Dictionary](https://github.com/Booleans/consumer-loan-survival-analysis/blob/master/data/LCDataDictionary.xlsx?raw=true).

The LoanStats3a file also contains information on loan applications that were declined and never issued. I have discarded those rows of data as they are not relevant to predicting loan defaults. I have also removed the last 2 rows of every spreadsheet, as they contained aggregate information on the number of loans in the file.

In [1]:
import datetime
import pickle
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from datetime import datetime as dt
from io import BytesIO
import boto3
import multiprocessing as mp

%run src/columns.py
%run src/data-cleaning.py
%run src/feature-engineering.py

In [2]:
def load_data_from_s3(filename, format='csv'):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket='loan-analysis-data', Key=filename)
    data = obj['Body'].read()
    f = BytesIO(data)
    if format=='csv':
        df = pd.read_csv(f, low_memory=False)
    if format=='pkl.bz2':
        df = pd.read_pickle(f, compression='bz2')
    return df

Reading in raw data, for now just use the pickle

In [3]:
loans = pd.read_pickle('data/raw_dataframe.pkl.bz2')
#loans = loans.sample(n=30000)

In [4]:
loans = drop_loan_status(loans)
loans = drop_joint_applicant_loans(loans)
loans = fix_rate_cols(loans)
loans.dropna(subset=['issue_d'], inplace=True)
loans = fix_date_cols(loans)
loans = exclude_loans_before_2010(loans)
loans = clean_loan_term_col(loans)
loans = only_include_36_month_loans(loans)
loans = clean_employment_length(loans)
loans = create_missing_data_boolean_columns(loans)
loans = fill_nas(loans, value=-99)
loans = add_supplemental_rate_data(loans)
loans = create_rate_difference_cols(loans)
loans = create_months_since_earliest_cl_col(loans)
loans = change_data_types(loans)
loans.set_index('id', inplace=True)

At this point I will save the dataframe to use for exploratory data analysis. After that we will be creating dummy cols for categorical variables as well as dropping columns that are no longer necessary.

In [5]:
loans.to_pickle('data/cleaned_loans_for_EDA.pkl.bz2', compression='bz2')

In [6]:
loans = create_dummy_cols(loans)
loans = drop_unnecessary_cols(loans)

loans.to_pickle('data/cleaned_loans.pkl.bz2', compression='bz2')

#### Return on Investment Calculation for Completed Loans